In [1]:
import tensorflow as tf
import os
from utils.helper_utils import HelperUtils
from configuration import Configuration
from utils.testing_util import TestingUtil
from pathlib import Path
import time
from pandas.core.frame import DataFrame

In [2]:
def train_model(model_name, data_dir):
    print(f'Training model {model_name} using data from {data_dir}')
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=Configuration.validation_split,
        subset="training",
        seed=123,
        image_size=(Configuration.img_height, Configuration.img_width),
        batch_size=Configuration.batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=Configuration.validation_split,
        subset="validation",
        seed=123,
        image_size=(Configuration.img_height, Configuration.img_width),
        batch_size=Configuration.batch_size)

    model, class_names = HelperUtils.load_model(Configuration.existing_model_name)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(train_ds, validation_data=val_ds, epochs=Configuration.epochs)
    HelperUtils.save_model(model_name, model, class_names)
    return model, class_names

Train and test models

In [3]:
models = []
for data_directory in Path('D:\\Studia\\magisterka\\sem2\\IUI\\data\\subsets').glob('*'):
    model_name = str(data_directory).split(os.sep)[-1]
    start_time = time.time()
    model, class_names = train_model(model_name, data_directory)
    train_time = time.time() - start_time
    results = TestingUtil.test_with_test_set(model, class_names)
    correct = len(list(filter(lambda x: x[-1] == 'Yes', results)))
    total = len(results)
    models.append((model_name, data_directory, f'{correct}/{total}', '{:.2f}%'.format(correct/total * 100), train_time))

Training model 100percent using data from D:\Studia\magisterka\sem2\IUI\data\subsets\100percent
Found 76536 files belonging to 432 classes.
Using 61229 files for training.
Found 76536 files belonging to 432 classes.
Using 15307 files for validation.
Epoch 1/10
1914/1914 [==============================] - 95s 46ms/step - loss: 0.0466 - accuracy: 0.9925 - val_loss: 2.3364 - val_accuracy: 0.8793
Epoch 2/10
1914/1914 [==============================] - 72s 38ms/step - loss: 0.0438 - accuracy: 0.9928 - val_loss: 2.3204 - val_accuracy: 0.8741
Epoch 3/10
1914/1914 [==============================] - 72s 38ms/step - loss: 0.0454 - accuracy: 0.9927 - val_loss: 2.5445 - val_accuracy: 0.8773
Epoch 4/10
1914/1914 [==============================] - 72s 37ms/step - loss: 0.0439 - accuracy: 0.9927 - val_loss: 2.3648 - val_accuracy: 0.8837
Epoch 5/10
1914/1914 [==============================] - 72s 38ms/step - loss: 0.0459 - accuracy: 0.9927 - val_loss: 2.3687 - val_accuracy: 0.8833
Epoch 6/10
1914/1914

In [4]:
print(DataFrame(models, columns=['Model name', 'Data path', 'Test results', 'Percentage test results', 'Training time']))

   Model name                                          Data path Test results  \
0  100percent  D:\Studia\magisterka\sem2\IUI\data\subsets\100...      270/311   
1   10percent  D:\Studia\magisterka\sem2\IUI\data\subsets\10p...      276/311   
2   20percent  D:\Studia\magisterka\sem2\IUI\data\subsets\20p...      276/311   
3   25percent  D:\Studia\magisterka\sem2\IUI\data\subsets\25p...      257/311   
4   33percent  D:\Studia\magisterka\sem2\IUI\data\subsets\33p...      274/311   
5   50percent  D:\Studia\magisterka\sem2\IUI\data\subsets\50p...      273/311   
6    5percent  D:\Studia\magisterka\sem2\IUI\data\subsets\5pe...       12/311   

  Percentage test results  Training time  
0                  86.82%     754.001761  
1                  88.75%      94.098684  
2                  88.75%     172.455463  
3                  82.64%     209.893530  
4                  88.10%     270.268595  
5                  87.78%     394.071866  
6                   3.86%      56.010140  
